In [1]:
! pip install --quiet cfgrib xarray pandas pyarrow

In [37]:
endpoint = 'http://172.200.202.2:80'
access_key = 'O6UZCRYPS9OPWUD1DYF8'
secret_key = 'HnF34+7JkYFusmbU++Cv0/YLUiEpWGBRZTjmUsuu'
bucket_name='csnowdb'
schema_name='grib2'


from vastdb.config import QueryConfig
cfg = QueryConfig()
cfg.data_endpoints=[
    "http://172.200.202.1",
    "http://172.200.202.2",
    "http://172.200.202.3",
    "http://172.200.202.4",
    "http://172.200.202.5",
    "http://172.200.202.6",
    "http://172.200.202.7",
    "http://172.200.202.8",
]

# Query

In [38]:
# query code 
import io
import os
import pyarrow as pa
from pyarrow import csv as pa_csv
import pyarrow.parquet as pq
from io import StringIO
import numpy as np
import pandas as pd
import vastdb

def read_parquet(file_path):
    """Reads Parquet data from a file."""
    try:
        return pq.read_table(file_path)
    except Exception as e:
        raise RuntimeError(f"Error reading Parquet file: {e}") from e

def connect_to_vastdb(endpoint, access_key, secret_key):
    """Connects to VastDB."""
    try:
        session = vastdb.connect(endpoint=endpoint, access=access_key, secret=secret_key)
        print("Connected to VastDB")
        return session
    except Exception as e:
        raise RuntimeError(f"Failed to connect to VastDB: {e}") from e

def query_vastdb(session, bucket_name, schema_name, table_name):
    """Writes data to VastDB."""
    with session.transaction() as tx:
        bucket = tx.bucket(bucket_name)
        schema = bucket.schema(schema_name, fail_if_missing=False) or bucket.create_schema(schema_name)
        table = schema.table(table_name, fail_if_missing=False) or schema.create_table(table_name, pa_table.schema)

        return table.select().read_all()

In [39]:
session = connect_to_vastdb(endpoint, access_key, secret_key)

Connected to VastDB


## Query Meta

In [41]:
table_name='grib2_data_ingestion_log'

with session.transaction() as tx:
    bucket = tx.bucket(bucket_name)
    schema = bucket.schema(schema_name, fail_if_missing=False) or bucket.create_schema(schema_name)
    table = schema.table(table_name, fail_if_missing=False) or schema.create_table(table_name, pa_table.schema)

    df = table.select().read_all().to_pandas()

print("---")
print(f"Total files processed: {df.shape[0]}")
print("---")

print("Metadata sample:")
display(df.head())


---
Total files processed: 1947
---
Metadata sample:


,ingestion_id,source_url,filename,file_size_bytes,download_timestamp,...,download_duration_seconds,processing_duration_seconds,grib2_variables,ingestion_batch_id,status
0,CMC_glb_PRES_PVU_1.5_latlon.15x.15_2025060400_...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_PRES_PVU_1.5_latlon.15x.15_2025060400_...,6043144,2025-06-04 22:00:19.898216,...,1.527817,2.635631,['pres'],GEM Global 15km_batch_177_1749074419,SUCCESS
1,CMC_glb_PRES_PVU_1_latlon.15x.15_2025060400_P0...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_PRES_PVU_1_latlon.15x.15_2025060400_P0...,6286927,2025-06-04 22:00:21.342441,...,1.381983,2.828120,['pres'],GEM Global 15km_batch_177_1749074419,SUCCESS
2,CMC_glb_PRES_PVU_2_latlon.15x.15_2025060400_P0...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_PRES_PVU_2_latlon.15x.15_2025060400_P0...,5887785,2025-06-04 22:00:21.426200,...,1.305320,2.639064,['pres'],GEM Global 15km_batch_177_1749074419,SUCCESS
3,CMC_glb_PRMSL_MSL_0_latlon.15x.15_2025060400_P...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_PRMSL_MSL_0_latlon.15x.15_2025060400_P...,838411,2025-06-04 22:01:18.314891,...,0.561108,2.006825,['prmsl'],GEM Global 15km_batch_178_1749074478,SUCCESS
4,CMC_glb_PRES_SFC_0_latlon.15x.15_2025060400_P0...,https://dd.weather.gc.ca/model_gem_global/15km...,CMC_glb_PRES_SFC_0_latlon.15x.15_2025060400_P0...,1187604,2025-06-04 22:01:18.314632,...,0.676665,2.040855,['sp'],GEM Global 15km_batch_178_1749074478,SUCCESS


## Query Data

In [42]:
table_name='grib2_data'

with session.transaction() as tx:
    bucket = tx.bucket(bucket_name)
    schema = bucket.schema(schema_name, fail_if_missing=False) or bucket.create_schema(schema_name)
    table = schema.table(table_name, fail_if_missing=False) or schema.create_table(table_name, pa_table.schema)

    df = table.select(limit_rows=10).read_all().to_pandas()
df

,latitude,longitude,time,step,isobaricInhPa,...,iprate,tirf,tsnowp,tmax,tmin
0,-90.0,-180.00,2025-06-04,0.0,200.0,...,NaN,NaN,NaN,NaN,NaN
1,-90.0,-179.85,2025-06-04,0.0,200.0,...,NaN,NaN,NaN,NaN,NaN
2,-90.0,-179.70,2025-06-04,0.0,200.0,...,NaN,NaN,NaN,NaN,NaN
3,-90.0,-179.55,2025-06-04,0.0,200.0,...,NaN,NaN,NaN,NaN,NaN
4,-90.0,-179.40,2025-06-04,0.0,200.0,...,NaN,NaN,NaN,NaN,NaN
5,-90.0,-179.25,2025-06-04,0.0,200.0,...,NaN,NaN,NaN,NaN,NaN
6,-90.0,-179.10,2025-06-04,0.0,200.0,...,NaN,NaN,NaN,NaN,NaN
7,-90.0,-178.95,2025-06-04,0.0,200.0,...,NaN,NaN,NaN,NaN,NaN
8,-90.0,-178.80,2025-06-04,0.0,200.0,...,NaN,NaN,NaN,NaN,NaN
9,-90.0,-178.65,2025-06-04,0.0,200.0,...,NaN,NaN,NaN,NaN,NaN


### Table stats

In [36]:
table_name='grib2_data'

with session.transaction() as tx:
    bucket = tx.bucket(bucket_name)
    schema = bucket.schema(schema_name, fail_if_missing=False) or bucket.create_schema(schema_name)
    table = schema.table(table_name, fail_if_missing=False) or schema.create_table(table_name, pa_table.schema)
    stats = table.stats
    
print("---")
print(f"Number of grib2 rows ingested into Vast DB: {stats.num_rows:,}")
print("---")

---
Number of grib2 rows ingested into Vast DB: 5,612,383,760
---


### Predicate pushdown

This cluster in 5.2 and only has semi-sorted projects.  Sorted tables are available with 5.4 which give log(N) query performance.

In [34]:
from ibis import _
from ibis.interactive import *

predicate=(
    (_.latitude == -90.0) & 
    (_.longitude == -180.00) &
    (_.valid_time == ibis.timestamp('2025-06-04 03:00:00')) &
    (~_.isobaricInhPa.isnull()) 
)

In [48]:
import time

table_name='grib2_data'

pd.set_option('display.width', 10000)
pd.set_option('display.max_columns', 10)
pd.set_option('display.show_dimensions', False)

start_time = time.perf_counter()
with session.transaction() as tx:
    bucket = tx.bucket(bucket_name)
    schema = bucket.schema(schema_name, fail_if_missing=False) or bucket.create_schema(schema_name)
    table = schema.table(table_name, fail_if_missing=False) or schema.create_table(table_name, pa_table.schema)

    print("querying table...")
    batchreader = table.select(predicate=predicate, limit_rows=10) # use limit_rows to simplify batch handling
    print("retrieved batch reader ...")

    try:
        for batch in batchreader:
            print(batch.to_pandas())
    except:
        pass
        
end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.4f} seconds")

querying table...
retrieved batch reader ...
   latitude  longitude       time  step  isobaricInhPa  ... iprate  tirf tsnowp tmax  tmin
0     -90.0     -180.0 2025-06-04   3.0          200.0  ...    NaN   NaN    NaN  NaN   NaN
   latitude  longitude       time  step  isobaricInhPa  ... iprate  tirf tsnowp tmax  tmin
0     -90.0     -180.0 2025-06-04   3.0          250.0  ...    NaN   NaN    NaN  NaN   NaN
   latitude  longitude       time  step  isobaricInhPa  ... iprate  tirf tsnowp tmax  tmin
0     -90.0     -180.0 2025-06-04   3.0          500.0  ...    NaN   NaN    NaN  NaN   NaN
   latitude  longitude       time  step  isobaricInhPa  ... iprate  tirf tsnowp tmax  tmin
0     -90.0     -180.0 2025-06-04   3.0          700.0  ...    NaN   NaN    NaN  NaN   NaN
   latitude  longitude       time  step  isobaricInhPa  ... iprate  tirf tsnowp tmax  tmin
0     -90.0     -180.0 2025-06-04   3.0          850.0  ...    NaN   NaN    NaN  NaN   NaN
   latitude  longitude       time  step  isob